In [1]:
import pandas as pd
import json
import regex as re
import sqlite3
import ast
import datetime
from contextlib import closing

In [2]:
# Function to parse json file after reformatting
# Returns parsed object or None if it could not be parsed
def parser(data):
    
    try: 
        output = ast.literal_eval(data)
    except SyntaxError:
        output = ""
    
    return output

# Function to reformat date
def reformatDate(date):
    
    if len(date) != 0:
        try:
            return datetime.datetime.strptime(date, '%d/%m/%Y').strftime('%Y-%m-%d')
        except ValueError:
            return datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d') 
        except TypeError:
            return None
    else:
        return None

# Function to trim overview to a max of 500 characters
def trimOverview(overview):
    overview = str(overview)
    return (overview[:497] + '...') if len(overview) > 500 else overview

In [3]:
# Input data source:
# https://www.kaggle.com/rounakbanik/the-movies-dataset

# Importing metadata csv to dataframe and dropping unneeded rows
metadata_dtype = {"adult": str, "budget": float, "id": int,"popularity": float, "video": str}

metadata = pd.read_csv("input/movies_metadata.csv", dtype=metadata_dtype, encoding='utf8', converters={'genres': parser, 'production_countries': parser, 'spoken_languages': parser, 'production_companies': parser, 'release_date': reformatDate, 'overview': trimOverview})

metadata.drop_duplicates(subset=['id'], inplace=True)

metadata.drop(labels=["belongs_to_collection", "imdb_id", "homepage", "adult", "poster_path", "video", "tagline", "original_title", 'vote_count', 'status', "popularity", "original_language"], inplace=True, axis=1, errors="ignore")

len(metadata)

45433

In [4]:
# Remove entries given condition
metadata = metadata[metadata['production_companies'].map(lambda x: x!=None and len(x)>0)]
metadata = metadata[metadata['production_countries'].map(lambda x: x!=None and len(x)>0)]
metadata = metadata[metadata['genres'].map(lambda x: x!=None and len(x)>0)]
metadata = metadata[metadata['spoken_languages'].map(lambda x: x!=None and len(x)>0)]
metadata = metadata[metadata['runtime'].map(lambda x: x>0)]
metadata = metadata[metadata['release_date'].map(lambda x: x!=None and len(x)>0)]
len(metadata)

31604

In [5]:
# Expanding json genres to id, genre rows
genres = metadata.filter(['id', 'genres'], axis=1)

# Flatten genres array
genres = genres.explode('genres')

# Create columns from genre dict
genres['genres'] = [row['name'] for row in genres['genres']]

genres.rename(columns={'id': 'movie_id', 'genres': 'genre_name'}, inplace=True)

genres

,movie_id,genre_name
0,862,Animation
0,862,Comedy
0,862,Family
1,8844,Adventure
1,8844,Fantasy
...,...,...
45457,324230,Comedy
45457,324230,Drama
45457,324230,Romance
45460,39922,Crime


In [6]:
moviecountries = metadata.filter(['id', 'production_countries'], axis=1)

# Flatten country array
moviecountries = moviecountries.explode('production_countries')

# Create columns from country dict
moviecountries['iso_3166_1'] = [row['iso_3166_1'] for row in moviecountries['production_countries']]
moviecountries['name'] = [row['name'] for row in moviecountries['production_countries']]

moviecountries.drop('production_countries', axis=1, inplace=True)

moviecountries.rename(columns={'id': 'movie_id', 'name': 'country_name', "iso_3166_1": "country_id"}, inplace=True)

countries = moviecountries.drop('movie_id', axis=1).drop_duplicates()


moviecountries.drop('country_name',inplace=True, axis=1)

moviecountries

,movie_id,country_id
0,862,US
1,8844,US
2,15602,US
3,31357,US
4,11862,US
...,...,...
45454,5422,IT
45455,169158,US
45457,324230,ES
45460,39922,DK


In [7]:
movielanguages = metadata.filter(['id', 'spoken_languages'], axis=1)

# Flatten languages array
movielanguages = movielanguages.explode('spoken_languages')

# Create columns from language dict
movielanguages['iso_639_1'] = [row['iso_639_1'] for row in movielanguages['spoken_languages']]
movielanguages['name'] = [row['name'] for row in movielanguages['spoken_languages']]

movielanguages.drop(labels=['spoken_languages'],inplace=True, axis=1)

movielanguages.rename(columns={'id': 'movie_id','name': 'language_name', 'iso_639_1':'language_id'}, inplace=True)

languages = movielanguages.drop('movie_id', axis=1).drop_duplicates()

movielanguages.drop('language_name',inplace=True, axis=1)

movielanguages

,movie_id,language_id
0,862,en
1,8844,en
1,8844,fr
2,15602,en
3,31357,en
...,...,...
45451,67758,en
45454,5422,it
45455,169158,en
45457,324230,es


In [8]:
moviecompanies = metadata.filter(['id', 'production_companies'], axis=1)

# Flatten companies array
moviecompanies = moviecompanies.explode('production_companies')

# Create columns from company dict
moviecompanies['name'] = [row['name'] for row in moviecompanies['production_companies']]
moviecompanies['company_id'] = [row['id'] for row in moviecompanies['production_companies']]

moviecompanies.rename(columns={'id': 'movie_id','name': 'company_name'}, inplace=True)

moviecompanies.drop(labels=['production_companies'],inplace=True, axis=1)

companies = moviecompanies.drop('movie_id', axis=1).drop_duplicates()

moviecompanies.drop(labels=['company_name'],inplace=True, axis=1)

moviecompanies

,movie_id,company_id
0,862,3
1,8844,559
1,8844,2550
1,8844,10201
2,15602,6194
...,...,...
45451,67758,6165
45454,5422,6117
45455,169158,4063
45457,324230,46055


In [9]:
# Importing metadata csv to dataframe and dropping unneeded rows
credits = pd.read_csv("input/credits.csv", encoding='utf8', converters={'cast': parser, 'crew': parser})

credits.drop_duplicates(subset=['id'], inplace=True)

credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [10]:
# Remove null cast
credits = credits[credits['cast'].map(lambda x: x!=None and x!=[])]
len(credits)

43018

In [11]:
cast = credits.filter(['id', 'cast'], axis=1)
# Remove any rows not in metadata
cast = cast[cast['id'].map(lambda x: (metadata['id'] == x).any())]

# Flatten cast arrays
cast = cast.explode('cast')

# Create columns from cast dict
cast['actor_id'] = [row['id'] for row in cast['cast']]
cast['character'] = [row['character'] for row in cast['cast']]
cast['actor_name'] = [row['name'] for row in cast['cast']]

# Remove null characters and actors
cast = cast[cast['character'].map(lambda x: x!=None and len(x)>0)]
cast = cast[cast['actor_name'].map(lambda x: x!=None and len(x)>0)]

# Create actor dataframe
actors = cast.filter(['actor_id', 'actor_name'], axis=1)
actors.drop_duplicates(subset=['actor_id'], inplace=True)

# Drop unused columns
cast.drop(labels=['cast', 'actor_name'],inplace=True, axis=1)

# Split characters into an array
cast['character'] = [[re.sub(r"^\W+", "", string) for string in row.split('/')] for row in cast['character']]
# Flatten characters array
cast = cast.explode('character')
cast.drop_duplicates(inplace=True)
len(cast)

455686

In [12]:
# Drop unused columns
metadata.drop(labels=['genres','production_countries', 'production_companies', 'spoken_languages'], inplace=True, axis=1, errors='ignore')

In [27]:
# Rename columns to match tables
metadata.rename(columns={'id': 'movie_id'}, inplace=True)
cast.rename(columns={'id': 'movie_id', 'character': 'character_name'}, inplace=True)

In [15]:
actors.drop_duplicates(subset=['actor_id'], inplace=True)

# Export dataframes to csv files
metadata.to_csv('output/metadata_clean.csv', index=False)
genres.to_csv('output/genres.csv', index=False)
countries.to_csv('output/production_countries.csv', index=False)
moviecountries.to_csv('output/movie_production_countries.csv', index=False)
companies.to_csv('output/production_companies.csv', index=False)
languages.to_csv('output/spoken_languages.csv', index=False)
movielanguages.to_csv('output/movie_spoken_languages.csv', index=False)
cast.to_csv('output/cast.csv', index=False)
actors.to_csv('output/actors.csv', index=False)

In [31]:
# For auto db connection cleanup
with closing(sqlite3.connect("db/movies.db")) as connection:
    with closing(connection.cursor()) as cursor:
        cursor = connection.cursor()

        # Create tables
        create_sql = open('db/create_movie_tables.sql').read()
        cursor.executescript(create_sql)
        
        # Insert data into tables
        metadata.to_sql('movie', connection, if_exists='append', index=False)
        genres.to_sql('genre', connection, if_exists='append', index=False)
        countries.to_sql('country', connection, if_exists='append', index=False)
        moviecountries.to_sql('moviecountry', connection, if_exists='append', index=False)
        companies.to_sql('company', connection, if_exists='append', index=False)
        moviecompanies.to_sql('moviecompany', connection, if_exists='append', index=False)
        actors.to_sql('actor', connection, if_exists='append', index=False)
        languages.to_sql('language', connection, if_exists='append', index=False)
        movielanguages.to_sql('movielanguage', connection, if_exists='append', index=False)
        cast.to_sql('character', connection, if_exists='append', index=False)
        print("Total changes: {}".format(connection.total_changes))

IntegrityError: FOREIGN KEY constraint failed